In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.base import clone
import os
from datetime import datetime
from sklearn import metrics
import seaborn as sn

In [2]:
# CONTROLS

#model = LogisticRegression(max_iter=10000)
#model = RandomForestClassifier()
model = xgb.XGBClassifier(eval_metric='error', use_label_encoder=False)

#missingness_threshold = 0.9
#missingness_threshold = 0.75
#missingness_threshold = 0.5
missingness_threshold = 0.25

#outcome = 'alc_postlt'
outcome = 'harmfuldrink'

#imputation_strategy = '-1'
#imputation_strategy = 'mode'
imputation_strategy = None # only for XGBoost

patient_questions_only = True
#patient_questions_only = False

include_feature_selection = True
#include_feature_selection = False

validation_center = 1 # choose from: 1-12 inclusive

save_results = True
#save_results = False

In [ ]:
df = pd.read_stata('./data/psychosocial_data.dta') # read in questions
print(df.shape)
df.head(11)

In [ ]:
question_data = df.filter(regex='^q[0-9]{1,3}$',axis=1) # filter to columns containing the 199 questions asked
question_data = pd.concat([question_data, df[['id', 'nid']]], axis=1) # keep id and nid columns
question_data.shape

In [ ]:
non_patient_questions = ['q1','q2','q3','q4','q5','q6','q7','q8','q9','q10','q12','q22',
                         'q33','q71','q129','q130','q145','q198']
mixed_questions = ['q28','q70','q146'] # 0 and NaN mean missing value
special_mixed_questions = ['q27','q40'] # only NaN means missing value

if patient_questions_only:
    question_data.drop(columns=non_patient_questions+mixed_questions+special_mixed_questions, inplace=True)
else: # split mixed questions
    def mixed_question_splitter(row, q): # return 1 if question has non-zero and non-null value
        if np.isnan(row[q]) or row[q] == 0:
            return 0
        return 1
    
    for q in mixed_questions: 
        new_label = q + '_split'
        question_data[new_label] = df.apply(lambda row: mixed_question_splitter(row, q), axis=1)
        
    def special_mixed_question_splitter(row, q): # return 1 if question has only non-null value
        if np.isnan(row[q]):
            return 0
        return 1
        
    for q in special_mixed_questions: 
        new_label = q + '_split'
        question_data[new_label] = df.apply(lambda row: special_mixed_question_splitter(row, q), axis=1)
    
question_data.shape

In [ ]:
question_data.head()

In [ ]:
# visualize missingness

question_data_unlabeled = question_data.drop(columns=['id', 'nid'])
q_nullseries = question_data_unlabeled.isna().sum() / question_data_unlabeled.shape[0] # calculate % NaN per column
ax = q_nullseries.plot.bar(x='question', y='% missing', rot=50, figsize=(50,20)) # plot question vs. % missing

In [ ]:
# collapse rows with the same ID by picking the one with the least missingness

for i in question_data.id.unique(): # for each unique id...
    temp = question_data[question_data.id == i].copy() # extract rows with corresponding id
    if temp.shape[0] > 1: # if there is more than one entry...
        idx = temp.isna().sum(axis=1).idxmin() # get the id of the row with min missing
        idxs = list(temp.index.values)
        idxs.remove(idx)
        question_data.drop(index=idxs, inplace=True)
        
question_data.shape

In [ ]:
# drop columns with a higher % missing than our threshold

cols_to_drop = []
    
for ind,val in q_nullseries.iteritems():
    if val > missingness_threshold:
        cols_to_drop.append(ind)

question_data.drop(columns=cols_to_drop, inplace=True)
question_data.shape

In [ ]:
#outcomes = pd.read_stata('./data/full_cohort_with_clinical_outcomes.dta') # older version of outcome data
outcomes = pd.read_stata('./data/test.dta') # read outcome data
print(outcomes.shape)
outcomes.head()

In [ ]:
outcomes = outcomes[[outcome, 'ptid', 'center', 'hospdeath']] # extract outcome, location, and ID data
outcomes.head()

In [ ]:
combined = question_data.merge(outcomes, left_on='id', right_on='ptid', how='left') # merge with question data
print(combined.shape)
combined.head()

In [ ]:
combined = combined[combined.hospdeath != 1] # remove patients who died in the hospital
combined.drop(columns=['hospdeath'], inplace=True)
combined.shape

In [ ]:
combined[outcome].value_counts() / combined.shape[0] # outcome distribution (percentage)

In [ ]:
combined[outcome].value_counts() # outcome distribution (raw counts)

In [ ]:
combined[outcome].isna().sum()

In [ ]:
# combined[outcome].fillna(0, inplace=True) # replace missing outcome with 0 meaning no harmful drinking
combined.drop(inplace=True, columns=['id', 'ptid', 'nid']) # remove merging variables
print(combined.shape)
combined.head()

In [ ]:
combined.to_csv('./data/data_final.csv')

In [ ]:
# reserve one center for validation

val = combined[combined.center == validation_center] # extract center for validation
val_X = val.drop(columns=[outcome]) # extract predictors
val_y = val[outcome] # extract outcome
print(val_X.shape, val_y.shape)

combined = combined[combined.center != validation_center] # remove center from rest of dataset

In [ ]:
X = combined.drop(columns=[outcome]) # extract predictors
y = combined[outcome] # extract outcome
print(X.shape, y.shape)

In [ ]:
X.isna().sum().sum()

In [ ]:
# impute missing data

if imputation_strategy == '-1':
    X.fillna(-1, inplace=True) # replace NaN with -1 to signify ommitted question
elif imputation_strategy == 'mode':
    for col in X.columns:
        X[col].fillna(X[col].mode()[0], inplace=True)

In [ ]:
X.isna().sum().sum()

In [ ]:
y.isna().sum()

In [ ]:
def forward_feature_selection(model, X, y):
    features = list(X.columns)
    selected_features = []
    scores = []
    
    for i in tqdm(range(X.shape[1])):
        best_score = 0
        next_feat = ''
        for feat in features:
            selected_features.append(feat)
            temp_X = X[selected_features]
            temp_scores = cross_validate(clone(model), temp_X, y, cv=5, scoring=['recall'])
            temp_score = temp_scores['test_recall'].mean()
            if temp_score >= best_score:
                best_score = temp_score
                next_feat = feat
            selected_features.pop()
        #print('Added Feature:', next_feat)
        selected_features.append(next_feat)
        features.remove(next_feat)
        scores.append(best_score)
        
    #print('Ordering of Features:', selected_features)
    
    plt.title('Forward Feature Selection')
    plt.xlabel('# of Features')
    plt.ylabel('recall')
    plt.plot(list(range(X.shape[1])), scores)
    return scores, selected_features

if include_feature_selection:
    scores, selected_features = forward_feature_selection(model, X, y)
    optimal_n_features = scores.index(max(scores)) + 1
    print('\nOptimal number of features:', optimal_n_features)
    optimal_features = selected_features[:optimal_n_features]
    X = X[optimal_features]
    val_X = val_X[optimal_features]

In [ ]:
scores = cross_validate(clone(model), X, y, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])

print('Accuracy:', round(scores['test_accuracy'].mean(), 4), "+/-", round(scores['test_accuracy'].std()*2, 4))
print('Precision/PPV:', round(scores['test_precision'].mean(), 4), "+/-", round(scores['test_precision'].std()*2, 4))
print('Recall:', round(scores['test_recall'].mean(), 4), "+/-", round(scores['test_recall'].std()*2, 4))
print('AUC:', round(scores['test_roc_auc'].mean(), 4), "+/-", round(scores['test_roc_auc'].std()*2, 4))

In [ ]:
# get correlation matrix of selected features

corrMatrix = X.corr()
sn.heatmap(corrMatrix, annot=True)
plt.rcParams['figure.figsize'] = 15, 15
plt.title('Correlation Matrix: Question Data')
plt.show()

In [ ]:
for feat in X.columns:
    print('--' + feat + '--')
    try:
        print('NaN ', q_nullseries[feat])
    except KeyError: # center variable not in q_nullseries, default to 0 missing values
        print('NaN  0')
    print(X[feat].value_counts(normalize=True)) # get value counts for our selected features for qualitative analysis

In [ ]:
model.fit(X, y)

In [ ]:
# have model predict on validation center to gague performance

val_y_pred = model.predict(val_X)
try:
    print('Accuracy:', round(metrics.accuracy_score(val_y, val_y_pred), 4))
    print('Precision/PPV:', round(metrics.precision_score(val_y, val_y_pred), 4))
    print('Recall', round(metrics.recall_score(val_y, val_y_pred), 4))
    print('AUC:', round(metrics.roc_auc_score(val_y, val_y_pred), 4))
except ValueError as e:
    print(e)

In [ ]:
# collect feature importance values

try:
    y = model.feature_importances_ # for Random Forest & XGBoost
except AttributeError:
    y = model.coef_[0] # for logistic regression

d = {'q': X.columns, 'feat_imp': y} # create feature importance dataframe
feat_imp = pd.DataFrame(data=d).sort_values(by='feat_imp', ascending=False)

In [ ]:
# merge % missing with feature importances

feat_imp = feat_imp.merge(q_nullseries.rename('% missing').to_frame(), left_index=False, left_on='q', right_index=True)

In [ ]:
def question_decoder(question_code):
    if '_split' in question_code:
        question_code = question_code.replace('_split','')
    question_code = question_code.upper()
    
    df = pd.read_excel('./data/Psychosocial data dictionary.xlsx', index_col=0)
    questions = df['QUESTIONS']
    try:
        return questions[question_code]
    except KeyError:
        return None

feat_imp['q_decoded'] = feat_imp['q'].apply(lambda x: question_decoder(x))
feat_imp

In [ ]:
if save_results:
    now = datetime.now()
    current_time = now.strftime("(%H:%M:%S)")
    current_date = now.date().strftime("%b-%d")
    
    filename = current_date + current_time + ".html"
    print("saving results to " + filename)
    os.system("ipython nbconvert --to html mvp.ipynb")
    os.rename('mvp.html', "./raw_results/" + filename)